# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 27 2022 1:03PM,240542,20,SSNA-2173694364,Sartorius Stedim North America,Released
1,Jun 27 2022 12:59PM,240540,20,ALUR002936881,Alumier Labs Inc.,Released
2,Jun 27 2022 12:59PM,240540,20,ALUR039209135,Alumier Labs Inc.,Released
3,Jun 27 2022 12:59PM,240540,20,ALUR055376002,Alumier Labs Inc.,Released
4,Jun 27 2022 12:59PM,240540,20,ALUR205453764,Alumier Labs Inc.,Released
5,Jun 27 2022 12:59PM,240540,20,ALUR243865393,Alumier Labs Inc.,Released
6,Jun 27 2022 12:59PM,240540,20,ALUR251553677,Alumier Labs Inc.,Released
7,Jun 27 2022 12:59PM,240540,20,ALUR265543201,Alumier Labs Inc.,Released
8,Jun 27 2022 12:59PM,240540,20,ALUR295914675,Alumier Labs Inc.,Released
9,Jun 27 2022 12:59PM,240540,20,ALUR300367137,Alumier Labs Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
43,240535,Released,1
44,240536,Released,1
45,240537,Released,9
46,240540,Released,34
47,240542,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
240535,NaN,1.0
240536,NaN,1.0
240537,NaN,9.0
240540,NaN,34.0
240542,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
240470,0.0,50.0
240471,0.0,57.0
240475,5.0,13.0
240476,1.0,0.0
240492,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
240470,0,50
240471,0,57
240475,5,13
240476,1,0
240492,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,240470,0,50
1,240471,0,57
2,240475,5,13
3,240476,1,0
4,240492,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,240470,,50
1,240471,,57
2,240475,5,13
3,240476,1,
4,240492,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America
1,Jun 27 2022 12:59PM,240540,20,Alumier Labs Inc.
35,Jun 27 2022 12:46PM,240537,12,Hush Hush
44,Jun 27 2022 12:41PM,240536,20,Alumier Labs Inc.
45,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc."
46,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation
96,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc."
97,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc."
98,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc."
99,Jun 27 2022 12:13PM,240522,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America,,1
1,Jun 27 2022 12:59PM,240540,20,Alumier Labs Inc.,,34
2,Jun 27 2022 12:46PM,240537,12,Hush Hush,,9
3,Jun 27 2022 12:41PM,240536,20,Alumier Labs Inc.,,1
4,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",,1
5,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation,,50
6,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc.",,1
7,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc.",,1
8,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc.",,1
9,Jun 27 2022 12:13PM,240522,10,ISDIN Corporation,,10


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America,1,
1,Jun 27 2022 12:59PM,240540,20,Alumier Labs Inc.,34,
2,Jun 27 2022 12:46PM,240537,12,Hush Hush,9,
3,Jun 27 2022 12:41PM,240536,20,Alumier Labs Inc.,1,
4,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1,
5,Jun 27 2022 12:37PM,240533,10,ISDIN Corporation,50,
6,Jun 27 2022 12:33PM,240534,21,"NBTY Global, Inc.",1,
7,Jun 27 2022 12:32PM,240532,10,"Methapharm, Inc.",1,
8,Jun 27 2022 12:19PM,240531,22,"NBTY Global, Inc.",1,
9,Jun 27 2022 12:13PM,240522,10,ISDIN Corporation,10,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America,1,
1,Jun 27 2022 12:59PM,240540,20,Alumier Labs Inc.,34,
2,Jun 27 2022 12:46PM,240537,12,Hush Hush,9,
3,Jun 27 2022 12:41PM,240536,20,Alumier Labs Inc.,1,
4,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America,1.0,NaN
1,Jun 27 2022 12:59PM,240540,20,Alumier Labs Inc.,34.0,NaN
2,Jun 27 2022 12:46PM,240537,12,Hush Hush,9.0,NaN
3,Jun 27 2022 12:41PM,240536,20,Alumier Labs Inc.,1.0,NaN
4,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 27 2022 1:03PM,240542,20,Sartorius Stedim North America,1.0,0.0
1,Jun 27 2022 12:59PM,240540,20,Alumier Labs Inc.,34.0,0.0
2,Jun 27 2022 12:46PM,240537,12,Hush Hush,9.0,0.0
3,Jun 27 2022 12:41PM,240536,20,Alumier Labs Inc.,1.0,0.0
4,Jun 27 2022 12:38PM,240535,21,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2886124,226.0,14.0
12,481042,10.0,1.0
15,962021,14.0,30.0
16,480999,1.0,1.0
19,481020,1.0,24.0
20,1924151,101.0,6.0
21,1683561,5.0,2.0
22,481056,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2886124,226.0,14.0
1,12,481042,10.0,1.0
2,15,962021,14.0,30.0
3,16,480999,1.0,1.0
4,19,481020,1.0,24.0
5,20,1924151,101.0,6.0
6,21,1683561,5.0,2.0
7,22,481056,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,226.0,14.0
1,12,10.0,1.0
2,15,14.0,30.0
3,16,1.0,1.0
4,19,1.0,24.0
5,20,101.0,6.0
6,21,5.0,2.0
7,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,226.0
1,12,Released,10.0
2,15,Released,14.0
3,16,Released,1.0
4,19,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,14.0,1.0,30.0,1.0,24.0,6.0,2.0,0.0
Released,226.0,10.0,14.0,1.0,1.0,101.0,5.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,14.0,1.0,30.0,1.0,24.0,6.0,2.0,0.0
1,Released,226.0,10.0,14.0,1.0,1.0,101.0,5.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,14.0,1.0,30.0,1.0,24.0,6.0,2.0,0.0
1,Released,226.0,10.0,14.0,1.0,1.0,101.0,5.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()